### Find Word Group Youtube Link

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 10000  # 28 native word end index

# youtube
sample_num = 10  # 7
time_shift = 0.6

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
            #df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].tail(sample_num)  # will test
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")\n
    df_search_result is dataframe and "search_string", "start_time", "end_time", "sentence", "video_id" are its columns
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [7]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [27]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 4/Result/2-Find Word Group Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

In [82]:
path_folder_file = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 4/Data/Deployment/Talk Time 4 Master File.xlsx"
sheet = "Sheet6"  # Sheet1
file_ext = "Sixgram"

In [83]:
df_file = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
#df_file = pd.read_excel(f"twogram.xlsx")
df_file

,6Gram
0,sana söylemem gereken bir şey var
1,söylemek istediğin bir şey var mı
2,daha iyi bir fikrin var mı
3,bunun iyi bir fikir olduğunu sanmıyorum
4,sana söylemek istediğim bir şey var
5,sana bir şey ifade ediyor mu
6,onu en son ne zaman gördün
7,sana göstermek istediğim bir şey var
8,söylemek istediğin bir şey mi var
9,bu iyi bir şey değil mi


In [84]:
search_list = df_file.iloc[:,0].to_list()
len(search_list)

29

In [85]:
#disable_video_id_list = ["H6E6N70jYqI","BoaYsdPtJYA","HUZINsU40Fk","Et3diPcEmfY"]
disable_video_id_list = []

In [86]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [87]:
df_youtube_sentence = df_youtube_sentence[~df_youtube_sentence["video_id"].isin(disable_video_id_list)]
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [88]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [89]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [90]:
df_word_group = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
df_word_group

,search_string,start_time,end_time,sentence,video_id
0,sana söylemem gereken bir şey var,1502.802,1506.201,hiçbiri umurumda değil sana söylemem gereken b...,pWZnIAh9MXo
1,sana söylemem gereken bir şey var,1987.900,1990.040,benim sana söylemem gereken bir şey var,KrLItbrSnZs
2,sana söylemem gereken bir şey var,1185.982,1187.865,sana söylemem gereken bir şey var benim,YN6xuSz86PA
3,sana söylemem gereken bir şey var,6182.280,6183.889,sana söylemem gereken bir şey var,4RMoZo69RE8
4,sana söylemem gereken bir şey var,1286.457,1288.660,sana söylemem gereken bir şey var,qe8TjfIqZ8Y
...,...,...,...,...,...
131,sana sormak istediğim bir şey var,4203.320,4205.193,benim sana sormak istediğim bir şey var ahmet,uU7R2ozdts4
132,sana sormak istediğim bir şey var,10006.071,10008.602,sana sormak istediğim bir şey var,OuRTW3NCa90
133,sana sormak istediğim bir şey var,3482.129,3484.708,niye geldin sana sormak istediğim bir şey var,vY7Q1i03tlg
134,sana sormak istediğim bir şey var,1515.680,1520.440,yani benim aslında söylemekten ziyade sana sor...,aMGS7vyx4Zo


In [91]:
df_word_group_time_loc = word_group_time_loc(df_word_group, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,sana söylemem gereken bir şey var,1504.137321,1506.201000,hiçbiri umurumda değil sana söylemem gereken b...,pWZnIAh9MXo
1,sana söylemem gereken bir şey var,1988.174359,1990.040000,benim sana söylemem gereken bir şey var,KrLItbrSnZs
2,sana söylemem gereken bir şey var,1185.982000,1187.623590,sana söylemem gereken bir şey var benim,YN6xuSz86PA
3,sana söylemem gereken bir şey var,6182.280000,6183.889000,sana söylemem gereken bir şey var,4RMoZo69RE8
4,sana söylemem gereken bir şey var,1286.457000,1288.660000,sana söylemem gereken bir şey var,qe8TjfIqZ8Y
...,...,...,...,...,...
131,sana sormak istediğim bir şey var,4203.528111,4204.984889,benim sana sormak istediğim bir şey var ahmet,uU7R2ozdts4
132,sana sormak istediğim bir şey var,10006.071000,10008.602000,sana sormak istediğim bir şey var,OuRTW3NCa90
133,sana sormak istediğim bir şey var,3482.759422,3484.708000,niye geldin sana sormak istediğim bir şey var,vY7Q1i03tlg
134,sana sormak istediğim bir şey var,1518.160563,1520.440000,yani benim aslında söylemekten ziyade sana sor...,aMGS7vyx4Zo


In [92]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,sana söylemem gereken bir şey var,1503.537321,1506.801000,hiçbiri umurumda değil sana söylemem gereken b...,pWZnIAh9MXo
1,sana söylemem gereken bir şey var,1987.574359,1990.640000,benim sana söylemem gereken bir şey var,KrLItbrSnZs
2,sana söylemem gereken bir şey var,1185.382000,1188.223590,sana söylemem gereken bir şey var benim,YN6xuSz86PA
3,sana söylemem gereken bir şey var,6181.680000,6184.489000,sana söylemem gereken bir şey var,4RMoZo69RE8
4,sana söylemem gereken bir şey var,1285.857000,1289.260000,sana söylemem gereken bir şey var,qe8TjfIqZ8Y
...,...,...,...,...,...
131,sana sormak istediğim bir şey var,4202.928111,4205.584889,benim sana sormak istediğim bir şey var ahmet,uU7R2ozdts4
132,sana sormak istediğim bir şey var,10005.471000,10009.202000,sana sormak istediğim bir şey var,OuRTW3NCa90
133,sana sormak istediğim bir şey var,3482.159422,3485.308000,niye geldin sana sormak istediğim bir şey var,vY7Q1i03tlg
134,sana sormak istediğim bir şey var,1517.560563,1521.040000,yani benim aslında söylemekten ziyade sana sor...,aMGS7vyx4Zo


In [93]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: round(x))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: round(x))
df_word_group_time_loc 

,search_string,start_time,end_time,sentence,video_id
0,sana söylemem gereken bir şey var,1504,1507,hiçbiri umurumda değil sana söylemem gereken b...,pWZnIAh9MXo
1,sana söylemem gereken bir şey var,1988,1991,benim sana söylemem gereken bir şey var,KrLItbrSnZs
2,sana söylemem gereken bir şey var,1185,1188,sana söylemem gereken bir şey var benim,YN6xuSz86PA
3,sana söylemem gereken bir şey var,6182,6184,sana söylemem gereken bir şey var,4RMoZo69RE8
4,sana söylemem gereken bir şey var,1286,1289,sana söylemem gereken bir şey var,qe8TjfIqZ8Y
...,...,...,...,...,...
131,sana sormak istediğim bir şey var,4203,4206,benim sana sormak istediğim bir şey var ahmet,uU7R2ozdts4
132,sana sormak istediğim bir şey var,10005,10009,sana sormak istediğim bir şey var,OuRTW3NCa90
133,sana sormak istediğim bir şey var,3482,3485,niye geldin sana sormak istediğim bir şey var,vY7Q1i03tlg
134,sana sormak istediğim bir şey var,1518,1521,yani benim aslında söylemekten ziyade sana sor...,aMGS7vyx4Zo


In [94]:
df_word_group_time_loc["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc['video_id'].map(str)+"&t="+df_word_group_time_loc['start_time'].map(str)+"s"
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id,video_url
0,sana söylemem gereken bir şey var,1504,1507,hiçbiri umurumda değil sana söylemem gereken b...,pWZnIAh9MXo,https://www.youtube.com/watch?v=pWZnIAh9MXo&t=...
1,sana söylemem gereken bir şey var,1988,1991,benim sana söylemem gereken bir şey var,KrLItbrSnZs,https://www.youtube.com/watch?v=KrLItbrSnZs&t=...
2,sana söylemem gereken bir şey var,1185,1188,sana söylemem gereken bir şey var benim,YN6xuSz86PA,https://www.youtube.com/watch?v=YN6xuSz86PA&t=...
3,sana söylemem gereken bir şey var,6182,6184,sana söylemem gereken bir şey var,4RMoZo69RE8,https://www.youtube.com/watch?v=4RMoZo69RE8&t=...
4,sana söylemem gereken bir şey var,1286,1289,sana söylemem gereken bir şey var,qe8TjfIqZ8Y,https://www.youtube.com/watch?v=qe8TjfIqZ8Y&t=...
...,...,...,...,...,...,...
131,sana sormak istediğim bir şey var,4203,4206,benim sana sormak istediğim bir şey var ahmet,uU7R2ozdts4,https://www.youtube.com/watch?v=uU7R2ozdts4&t=...
132,sana sormak istediğim bir şey var,10005,10009,sana sormak istediğim bir şey var,OuRTW3NCa90,https://www.youtube.com/watch?v=OuRTW3NCa90&t=...
133,sana sormak istediğim bir şey var,3482,3485,niye geldin sana sormak istediğim bir şey var,vY7Q1i03tlg,https://www.youtube.com/watch?v=vY7Q1i03tlg&t=...
134,sana sormak istediğim bir şey var,1518,1521,yani benim aslında söylemekten ziyade sana sor...,aMGS7vyx4Zo,https://www.youtube.com/watch?v=aMGS7vyx4Zo&t=...


In [95]:
word_count_result(df_word_group_time_loc, ["search_string"], set_condition=True)

,word,word_count
0,bir,19
1,şey,17
2,var,15
3,mı,7
4,söylemek,5
5,iyi,4
6,istediğin,4
7,gereken,3
8,yapabileceğim,3
9,sana,3


In [96]:
df_word_group_time_loc.to_excel(f"{lang_folder.capitalize()}_Word_Group_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result_{file_ext}.xlsx", index=False) 

#### Copy Move And Delete

In [97]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result_{file_ext}.xlsx")
output_file

['Turkish_Word_Group_With_10000_Word_10_Youtube_0.6s_Timeshift_For_Talk_Time_Result_Sixgram.xlsx']

In [98]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [99]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass